In [14]:
import matplotlib.pyplot as plt
import pandas
import numpy as np
import random

# gerrychain imports
from gerrychain import (GeographicPartition, Partition, Graph, MarkovChain,
                        proposals, updaters, constraints, accept, Election)
from gerrychain.proposals import recom
from gerrychain.tree import recursive_tree_part
from gerrychain.accept import always_accept
from gerrychain.constraints import (Validator, single_flip_contiguous,
                                    within_percent_of_ideal_population, UpperBound)
from functools import partial

In [15]:
# State specific variables
state = "LA" # Louisiana
num_districts = 39 # senate districts
data_path = "./LA_data/LA_data.json"
pop_varname = "TOTPOP"
bvap_varname = "BVAP"
vap_varname = "VAP"
num_markov_steps = 20000 # in Markov chain
OPPORTUNITY_PERCENT = 0.4

In [16]:
# load the graph
graph = Graph.from_json(data_path)

In [17]:
#################################################################################
########################## Get ideal population  ################################
#################################################################################

def get_ideal_population(graph, pop_varname, num_districts):
    tot_pop = 0
    for i in range(len(graph.nodes)):
        tot_pop += graph.nodes[i][pop_varname]
        
    return int(tot_pop/num_districts)

ideal_population = get_ideal_population(graph, pop_varname, num_districts)

####################################################################################
############################# Define the updaters ##################################
####################################################################################

def count_opportunity_districts(partition):
    opp_percent = OPPORTUNITY_PERCENT
    
    num_opp_dists = 0
    for i in range(num_districts):
        num_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        
        if num_bvap_percent >= opp_percent:
            num_opp_dists += 1

    return num_opp_dists

def compute_funky_score(partition):
    curr_bvap_list = []
    
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        curr_bvap_list.append(curr_bvap_percent)
    
    curr_bvap_list = sorted(curr_bvap_list)
    
    curr_funky_score = 0
    for i in range(num_districts-1,-1,-1):
        if curr_bvap_list[i] >= OPPORTUNITY_PERCENT:
            curr_funky_score += 1
        else:
            curr_funky_score += curr_bvap_list[i]
            break
    
    return curr_funky_score

####################################################################################
###################### End of updater definitions ##################################
####################################################################################

updaters = {"population": updaters.Tally(pop_varname, alias="population"),
            "bvap":updaters.Tally(bvap_varname, alias='bvap'),
            "vap":updaters.Tally(vap_varname, alias='vap'),
            "opportunity_districts":count_opportunity_districts,
            "funky_score": compute_funky_score
           }

In [18]:
####################################################################################
################ Build Initial Partition and Markov Chain ##########################
####################################################################################

# assign each node to a district in a dict of the form {node: district}
seed_partition_dict = recursive_tree_part(graph, 
                                          range(num_districts), 
                                          pop_col=pop_varname,
                                          pop_target=ideal_population,
                                          epsilon=0.01, 
                                          node_repeats=1)

# use the partition dictionary to build a Partition object
seed_partition = GeographicPartition(graph, 
                                     assignment=seed_partition_dict, 
                                     updaters=updaters)

## Build the Markov chain
tree_proposal = partial(recom,
                        pop_col=pop_varname,
                        pop_target=ideal_population,
                        epsilon=0.05,
                        node_repeats=1)

# Constraints
popbound = within_percent_of_ideal_population(seed_partition, .1)

####################################################################################
###################### Define the acceptance functions #############################
####################################################################################

def increase_opportunity_districts(partition, bad_mobility_prob = 0.2):
    opp_percent = OPPORTUNITY_PERCENT
    if partition.parent is None:
        return True
    
    curr_opp_dists = 0
    prev_opp_dists = 0
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        prev_bvap_percent = partition.parent["bvap"][i] / partition.parent["vap"][i]
        
        if curr_bvap_percent >= opp_percent:
            curr_opp_dists += 1
        if prev_bvap_percent >= opp_percent:
            prev_opp_dists += 1
        
    if curr_opp_dists >= prev_opp_dists: 
        return True
    else:
        return random.random() < bad_mobility_prob
            

# Funky Weighted Opportunity Score
def increase_funky_opportunity_score(partition, bad_mobility_prob = 0.2):
    
    curr_bvap_list = []
    prev_bvap_list = []
    
    for i in range(num_districts):
        curr_bvap_percent = partition["bvap"][i] / partition["vap"][i]
        prev_bvap_percent = partition.parent["bvap"][i] / partition.parent["vap"][i]
        
        curr_bvap_list.append(curr_bvap_percent)
        prev_bvap_list.append(prev_bvap_percent)
    
    curr_bvap_list = sorted(curr_bvap_list)
    prev_bvap_list = sorted(prev_bvap_list)
    
    curr_bvap_score = 0
    for i in range(num_districts-1,-1,-1):
        if curr_bvap_list[i] >= OPPORTUNITY_PERCENT:
            curr_bvap_score += 1
        else:
            curr_bvap_score += curr_bvap_list[i]
            break
    
    prev_bvap_score = 0 
    for i in range(num_districts-1,-1,-1):
        if prev_bvap_list[i] >= OPPORTUNITY_PERCENT:
            prev_bvap_score += 1
        else:
            prev_bvap_score += prev_bvap_list[i]
            break
            
    if curr_bvap_score >= prev_bvap_score:   
        return True
    else:
        return random.random() < bad_mobility_prob
       

In [19]:
####################################################################################
################################## Short Burst #####################################
####################################################################################    

def short_burst_run(initial_partition, num_bursts, num_steps, accept_function, score_function):
    max_partition = initial_partition
    max_score = 0 # bhushan come back here later...
    
    scores = []

    for i in range(num_bursts):
        chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=accept_function,
                          initial_state=max_partition, 
                          total_steps=num_steps)

        for partition in chain:
            current_score = score_function(partition)
            if current_score >= max_score: # > or >= has implications, not sure which is best...
                max_partition = partition
                max_score = current_score
            
            scores.append(current_score)
            
        if i % 10 == 0:
            print("Burst no. ", i)
                
    return scores


In [20]:
burst_lengths = [1]
num_bursts = [int(num_markov_steps / burst_length) for burst_length in burst_lengths]

for i in range(len(burst_lengths)):
    print("Burst Length: ", burst_lengths[i], ", Num bursts: ", num_bursts[i])
    scores = short_burst_run(seed_partition, 
                             num_bursts[i], 
                             burst_lengths[i],
                             always_accept, 
                             count_opportunity_districts)
    
    # save array 
    scores = np.array(scores)
    np.save("op_dist_counts_bl_" + str(burst_lengths[i]), scores)

Burst Length:  1 , Num bursts:  20000
Burst no.  0
Burst no.  10
Burst no.  20
Burst no.  30
Burst no.  40
Burst no.  50
Burst no.  60
Burst no.  70
Burst no.  80
Burst no.  90
Burst no.  100
Burst no.  110
Burst no.  120
Burst no.  130
Burst no.  140
Burst no.  150
Burst no.  160
Burst no.  170
Burst no.  180
Burst no.  190
Burst no.  200
Burst no.  210
Burst no.  220
Burst no.  230
Burst no.  240
Burst no.  250
Burst no.  260
Burst no.  270
Burst no.  280
Burst no.  290
Burst no.  300
Burst no.  310
Burst no.  320
Burst no.  330
Burst no.  340
Burst no.  350
Burst no.  360
Burst no.  370
Burst no.  380
Burst no.  390
Burst no.  400
Burst no.  410
Burst no.  420
Burst no.  430
Burst no.  440
Burst no.  450
Burst no.  460
Burst no.  470
Burst no.  480
Burst no.  490
Burst no.  500
Burst no.  510
Burst no.  520
Burst no.  530
Burst no.  540
Burst no.  550
Burst no.  560
Burst no.  570
Burst no.  580
Burst no.  590
Burst no.  600
Burst no.  610
Burst no.  620
Burst no.  630
Burst no.  64

Burst no.  6190
Burst no.  6200
Burst no.  6210
Burst no.  6220
Burst no.  6230
Burst no.  6240
Burst no.  6250
Burst no.  6260
Burst no.  6270
Burst no.  6280
Burst no.  6290
Burst no.  6300
Burst no.  6310
Burst no.  6320
Burst no.  6330
Burst no.  6340
Burst no.  6350
Burst no.  6360
Burst no.  6370
Burst no.  6380
Burst no.  6390
Burst no.  6400
Burst no.  6410
Burst no.  6420
Burst no.  6430
Burst no.  6440
Burst no.  6450
Burst no.  6460
Burst no.  6470
Burst no.  6480
Burst no.  6490
Burst no.  6500
Burst no.  6510
Burst no.  6520
Burst no.  6530
Burst no.  6540
Burst no.  6550
Burst no.  6560
Burst no.  6570
Burst no.  6580
Burst no.  6590
Burst no.  6600
Burst no.  6610
Burst no.  6620
Burst no.  6630
Burst no.  6640
Burst no.  6650
Burst no.  6660
Burst no.  6670
Burst no.  6680
Burst no.  6690
Burst no.  6700
Burst no.  6710
Burst no.  6720
Burst no.  6730
Burst no.  6740
Burst no.  6750
Burst no.  6760
Burst no.  6770
Burst no.  6780
Burst no.  6790
Burst no.  6800
Burst no

Burst no.  12960
Burst no.  12970
Burst no.  12980
Burst no.  12990
Burst no.  13000
Burst no.  13010
Burst no.  13020
Burst no.  13030
Burst no.  13040
Burst no.  13050
Burst no.  13060
Burst no.  13070
Burst no.  13080
Burst no.  13090
Burst no.  13100
Burst no.  13110
Burst no.  13120
Burst no.  13130
Burst no.  13140
Burst no.  13150
Burst no.  13160
Burst no.  13170
Burst no.  13180
Burst no.  13190
Burst no.  13200
Burst no.  13210
Burst no.  13220
Burst no.  13230
Burst no.  13240
Burst no.  13250
Burst no.  13260
Burst no.  13270
Burst no.  13280
Burst no.  13290
Burst no.  13300
Burst no.  13310
Burst no.  13320
Burst no.  13330
Burst no.  13340
Burst no.  13350
Burst no.  13360
Burst no.  13370
Burst no.  13380
Burst no.  13390
Burst no.  13400
Burst no.  13410
Burst no.  13420
Burst no.  13430
Burst no.  13440
Burst no.  13450
Burst no.  13460
Burst no.  13470
Burst no.  13480
Burst no.  13490
Burst no.  13500
Burst no.  13510
Burst no.  13520
Burst no.  13530
Burst no.  135

Burst no.  18170
Burst no.  18180
Burst no.  18190
Burst no.  18200
Burst no.  18210
Burst no.  18220
Burst no.  18230
Burst no.  18240
Burst no.  18250
Burst no.  18260
Burst no.  18270
Burst no.  18280
Burst no.  18290
Burst no.  18300
Burst no.  18310
Burst no.  18320
Burst no.  18330
Burst no.  18340
Burst no.  18350
Burst no.  18360
Burst no.  18370
Burst no.  18380
Burst no.  18390
Burst no.  18400
Burst no.  18410
Burst no.  18420
Burst no.  18430
Burst no.  18440
Burst no.  18450
Burst no.  18460
Burst no.  18470
Burst no.  18480
Burst no.  18490
Burst no.  18500
Burst no.  18510
Burst no.  18520
Burst no.  18530
Burst no.  18540
Burst no.  18550
Burst no.  18560
Burst no.  18570
Burst no.  18580
Burst no.  18590
Burst no.  18600
Burst no.  18610
Burst no.  18620
Burst no.  18630
Burst no.  18640
Burst no.  18650
Burst no.  18660
Burst no.  18670
Burst no.  18680
Burst no.  18690
Burst no.  18700
Burst no.  18710
Burst no.  18720
Burst no.  18730
Burst no.  18740
Burst no.  187

In [ ]:
# Opportunity district cell
recom_chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=increase_opportunity_districts,
                          initial_state=seed_partition, 
                          total_steps=num_markov_steps)

opportunity_districts_counts = []

for partition in recom_chain:
    opportunity_districts_counts.append(partition["opportunity_districts"])
    
plt.plot(opportunity_districts_counts, range(num_markov_steps))
plt.title("Opportunity districts increase / remain same")
plt.xlabel("# of opportunity districts")
plt.ylabel("# of steps in chain")

# save array 
opportunity_districts_counts = np.array(opportunity_districts_counts)
np.save("opportunity_district_counts", opportunity_districts_counts)

In [ ]:
# Funky score cell
recom_chain = MarkovChain(tree_proposal, 
                          Validator([popbound]), 
                          accept=increase_funky_opportunity_score,
                          initial_state=seed_partition, 
                          total_steps=num_markov_steps)

funky_scores = []

for partition in recom_chain:    
    funky_scores.append(partition["funky_score"])      

plt.plot(funky_scores, range(num_markov_steps))
plt.title("Funky Scores increase / remain same")
plt.xlabel("Funky Score")
plt.ylabel("# of steps in chain")

# save array 
funky_scores = np.array(funky_scores)
np.save("funky_scores", funky_scores)